# <font color=blue size=5>Configurando o ambiente</font>

Importação de bibliotecas

In [ ]:
import mysql.connector as sql
import pandas as pd
import numpy as np

Conectando com a base local do MySQL no banco 'vendas_2020'

In [ ]:
db = sql.connect(
host = 'localhost',
user = 'root',
passwd = 'root',
database = 'vendas_2020')

Ingestão dos dados na variável vendas_janeiro

In [ ]:
vendas_janeiro = pd.read_sql('SELECT * FROM vendas_janeiro;', db)
db.close()

Visualização e remoção de linhas com dados ausentes

In [ ]:
vendas_janeiro.head(5);

In [ ]:
vendas_janeiro.isnull().sum();

In [ ]:
vendas_janeiro[vendas_janeiro.isna().any(axis=1)];

In [ ]:
vendas_janeiro = vendas_janeiro.dropna();

In [ ]:
vendas_janeiro.isnull().sum();

# <font color=blue size=5>Análise geral</font>

Informações básicas sobre o DataFrame

In [ ]:
vendas_janeiro.describe().round(2)

# <font color=blue size=5>Taxa de aprovação</font>

Taxa de aprovação geral dos pedidos para o mês de janeiro

In [ ]:
aprovados = vendas_janeiro.loc[vendas_janeiro['situacao']=='Aprovado']['situacao'].count()
tot_pedidos = vendas_janeiro['situacao'].count()
print('Taxa de aprovação:',((aprovados/tot_pedidos)*100).round(2),'%')

# <font color=blue size=5>Vendas por tipo e por canal</font>

Receita total por tipo de venda

In [ ]:
vendas_janeiro[['tipo_venda','receita_total']].groupby(['tipo_venda']).sum().round(2)

Receita total por canal de venda

In [ ]:
vendas_janeiro[['canal','receita_total']].groupby(['canal']).sum().sort_values(by=['receita_total'],ascending = False).round(2)

Receita total agrupada por tipo e canal de venda

In [ ]:
vendas_janeiro[['tipo_venda','canal','receita_total']].groupby(['tipo_venda','canal']).sum().sort_values(by=['tipo_venda','receita_total'],ascending = [1,0]).round(2)

# <font color=blue size=5>Frete por UF</font>

Valor total de receita oriunda do frete dos pedidos agrupado por estado

In [ ]:
vendas_janeiro[['entrega_uf','receita_frete']].groupby(['entrega_uf']).sum().sort_values(by='receita_frete', ascending = False).round(2)

# <font color=blue size=5>Receita média por categoria de produto</font>

Criação de uma view relacionando categoria, receita total e quantidade de produtos

In [ ]:
receita_por_categoria = vendas_janeiro[['categoria','receita_total','quantidade_produto']].groupby(['categoria']).sum().sort_values(by='receita_total', ascending = False)

Receita média superior ou igual a R$500 por categoria para quantidades de produto superiores ou iguais a 10

In [ ]:
receita_por_categoria['receita_media'] = receita_por_categoria['receita_total']/receita_por_categoria['quantidade_produto']
receita_por_categoria[(receita_por_categoria['quantidade_produto']>=10) & (receita_por_categoria['receita_media']>=500)].sort_values(by='receita_media', ascending = False).round(2)

# <font color=blue size=5>Volume de vendas por região</font>

Quantidade de vendas por estado no mês de janeiro

In [ ]:
vendas_janeiro.groupby('entrega_uf').size().reset_index(name='vendas').sort_values(by='vendas', ascending = False)

# <font color=blue size=5>Vendas e receita por dia</font>

Adição da coluna dia_compra na tabela

In [ ]:
dias = pd.to_datetime(vendas_janeiro['data_compra']).dt.day
vendas_janeiro['dia_compra'] = dias

Receita total por dia no mês de janeiro

In [ ]:
vendas_janeiro[['dia_compra','receita_total']].groupby('dia_compra').sum().round(2)

Vendas por dia no mês de janeiro

In [ ]:
vendas_janeiro.groupby('dia_compra').size().reset_index(name='vendas')

In [ ]:
vendas_janeiro.to_excel("Base Tratada.xlsx", index=False)